In [ ]:
import pandas as pd
import sqlite3

# Шаг 1 - Чтение публичного CSV файла в pandas

Шаг 1
(Балл - 0.2) Необходимо:
- скачать CSV-файл — «final_transactions.csv»,
- создать таблицу transaction_bd со всеми полями,
- загрузить данные из файла в таблицу и оставить таблицу со структурой:

      (0, 'TX_DATETIME', 'NUMERIC', 0, None, 0) - время транзакций
      (1, 'CUSTOMER_ID', 'INTEGER', 0, None, 0) - клиент
      (2, 'TX_AMOUNT', 'REAL', 0, None, 0) - сумма транзакций

Комментарий: Можно оставить все 5 полей, но запросы будут выполняться чуть дольше.

In [ ]:
# публичная ссылка на файл 'final_transactions.csv'
url_file = 'https://drive.google.com/file/d/1_YrZPaD1zNCcXUkUfeWM0ZOdg_was_iA/view'

# замена левой части адреса
main = 'https://drive.google.com/uc?id='

# Берем id файла 
id_url = url_file.split('/')[-2]

# url пригодный для чтения
direct_url = main + id_url

# cкачиваем и читаем файл
df = pd.read_csv(direct_url)
df.head()


## Создание пустой базы

In [ ]:
from sqlite3 import Error 
def create_connection(path):
    '''
    Подключаемся к имеющейся или создаем новую
    есл не найдена
    '''
    connection = None
    try:
        connection = sqlite3.connect(path)
        print("Connection to SQLite DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection


def execute_query(connection, query):
    """
    Передаем запрос в бакзу с созданным 
    ранее connection
    """
    cursor = connection.cursor()
    try:
        cursor.execute(query)
        connection.commit()
        print("Query executed successfully")
    except Error as e:
        print(f"The error '{e}' occurred")

In [ ]:
# создаем базу transaction_bd
data_base = 'Transaction_bd.db'
connection = create_connection(data_base)

## Создание таблицы в базе

In [ ]:
df.columns

In [ ]:
# создаем запрос на таблицу пустую
table = 'Transactions'
create_table = f"""
CREATE TABLE IF NOT EXISTS {table} (
  TRANSACTION_ID INTEGER PRIMARY KEY AUTOINCREMENT,
  TX_DATETIME NUMERIC NOT NULL,
  CUSTOMER_ID INTEGER,
  TERMINAL_I INTEGER,
  TX_AMOUNT REAL
);
"""

In [ ]:
# создаем в базе таблицу  Transactions
execute_query(connection, create_table)

## Наполнение таблицы  из датафрейма

In [ ]:
# создаем в базе таблицу  Transactions из датафрейма
df.to_sql(table, connection, if_exists='replace', index=False)

In [ ]:
# смотрим базу
sql_string = 'SELECT * FROM Transactions'
df_check = pd.read_sql(sql_string, connection,  index_col='TRANSACTION_ID')
df_check.tail()

## Удаление лишних колонок из таблицы базы SQL

In [ ]:
DROP_COLUMNS = ('TERMINAL_ID',) # TRANSACTION_ID
# удаляем столбцы лишние по заданию
for column in DROP_COLUMNS:  
    cursor = connection.cursor()
    alter_column = f'ALTER TABLE {table} DROP COLUMN {column}'
    cursor.execute(alter_column)
    connection.commit()

In [ ]:
# смотрим базу
sql_string = 'SELECT * FROM Transactions'
df_check = pd.read_sql(sql_string, connection, index_col='TRANSACTION_ID')
df_check.head()

# Шаг 2
(Балл - 0.1 за каждый пункт) Написать следующие запросы к таблице transaction_bd:

- A. Вывести всех клиентов, у которых сумма транзакций больше 700000 за весь период (сортируя клиентов по возрастанию);
- B. Вывести всех клиентов, у которых сумма транзакций больше 200000 за период 01.01.2023 - 13.01.2023 (сортируя клиентов по возрастанию);
- C. Вывести тех клиентов, у которых id начинается с 4 и количество транзакций за весь период более 444;
- D. Создать флаг доходности клиентов по логике:
    1. Если сумма транзакций не более 50000, тогда вывести 'низкая доходность';
    2. Если сумма транзакций больше 50000 и не более 10000 тогда вывести 'средняя доходность';
    3. Если сумма транзакций больше 100000 тогда вывести 'высокая доходность'.
- E. Посмотреть количество клиентов с каждым видом доходности (из пункта 2.d);
- F. Вывести сумму транзакций за каждый день (сортируя дни по возрастанию).


Оконные функции SQL простым языком с примерами - https://habr.com/ru/post/664000/

### A. Вывести всех клиентов, у которых сумма транзакций больше 700000 за весь период (сортируя клиентов по возрастанию);

In [ ]:
limit = 700000
query_a = f'''
SELECT CUSTOMER_ID
FROM {table}
GROUP BY CUSTOMER_ID
HAVING sum(TX_AMOUNT) > {limit}
ORDER BY CUSTOMER_ID ASC
'''
# получаем датафрейм по запросу
data = pd.read_sql(query_a, connection)
data

### B. Вывести всех клиентов, у которых сумма транзакций больше 200000 за период 01.01.2023 - 13.01.2023 (сортируя клиентов по возрастанию);

In [ ]:
limit = 200000
query_b = f'''
SELECT CUSTOMER_ID
FROM {table}
GROUP BY CUSTOMER_ID
HAVING sum(TX_AMOUNT) > {limit} AND
TX_DATETIME BETWEEN '2023-01-01' AND '2023-01-13'
ORDER BY CUSTOMER_ID ASC 
'''
# получаем датафрейм по запросу
data = pd.read_sql(query_b, connection)
data

### C. Вывести тех клиентов, у которых id начинается с 4 и количество транзакций за весь период более 444;

In [ ]:
limit = 444
query_c = f'''
SELECT CUSTOMER_ID
FROM {table}
where SUBSTRING(CUSTOMER_ID, 1, 1) = '4'
GROUP BY CUSTOMER_ID
HAVING sum(TX_AMOUNT) > {limit}
ORDER BY CUSTOMER_ID ASC 
'''
# получаем датафрейм по запросу
data = pd.read_sql(query_c, connection)
data

### D. Создать флаг доходности клиентов по логике:
    1. Если сумма транзакций не более 50000, тогда вывести 'низкая доходность';
    2. Если сумма транзакций больше 50000 и не более 100000 тогда вывести 'средняя доходность';
    3. Если сумма транзакций больше 100000 тогда вывести 'высокая доходность'.

In [ ]:
level_1 = 50000
level_2 = 100000
query_d = f'''
SELECT CUSTOMER_ID,
CASE WHEN sum(TX_AMOUNT) <= {level_1} THEN 'низкая доходность'
     WHEN sum(TX_AMOUNT) > {level_1} and sum(TX_AMOUNT) <= {level_2} THEN 'средняя доходность'
ELSE 'высокая доходность' 
END  RATE
FROM {table}
GROUP BY CUSTOMER_ID
ORDER BY CUSTOMER_ID ASC 

'''
# получаем датафрейм по запросу
data = pd.read_sql(query_d, connection)
data

### E. Посмотреть количество клиентов с каждым видом доходности (из пункта 2.d);

In [ ]:
# берем предыдущий запрос и вставляем его в новый
query_e = f'''
WITH TAB as ({query_d})
SELECT TAB.RATE, count(TAB.RATE) as QTY
FROM TAB
GROUP BY RATE
'''
# получаем датафрейм по запросу
data = pd.read_sql(query_e, connection)
data

### F. Вывести сумму транзакций за каждый день (сортируя дни по возрастанию).

In [ ]:
# берем предыдущий запрос и вставляем его в новый
query_f = f'''
SELECT SUBSTRING(TX_DATETIME, 1, 10) as DATE,
       sum(TX_AMOUNT) as SUMM
FROM  {table}
GROUP BY DATE
ORDER BY DATE ASC
'''
# получаем датафрейм по запросу
data_dash = pd.read_sql(query_f, connection)
data_dash

In [ ]:
connection.close()

# Шаг 3
(Балл - 0.2) 

- Подготовить дашборд с помощью Dash по пункту 2.f, включив туда графики bar и histogram;
- Bставить в конце ноутбука скрин графиков из дашборда.


## Подготовка дашборда

In [ ]:
#!pip install dash
#!pip install jupyter-dash

In [ ]:
from dash import Dash, html, dcc
import plotly.express as px

## Графики

Шпаргалка по визуализации данных в Python с помощью Plotly - https://habr.com/ru/post/502958/

In [ ]:
app = Dash()
title_1 = 'Сумма транзакций за каждый день'
title_2 = 'Распределение сумм транзакций в день'

fig1 = px.bar(data_dash, x="DATE", y="SUMM", title = title_1)

fig2 = px.histogram(data_dash, x = "SUMM",
             title = title_2,
             nbins = 100,
             )
fig1.update_layout(template = 'plotly_dark') # темная тема
fig2.update_layout(template = 'plotly_dark') # темная тема

# публикация на веб_сервере для браузера
app.layout = html.Div(children=[
    html.H1(children = title_1),

    html.Div(children = title_2),

    dcc.Graph(
        id='example-graph',
        figure=fig1
    ),
     dcc.Graph(
        id='example-graph1',
        figure=fig2
    )
])

app.run_server()

<img src="https://raw.githubusercontent.com/Mike030668/MIPT_magistratura/main/Data%20storage%20and%20processing%20systems/images/img-2023-03-29-20-57-12.png" alt="Screenshort" />